# Imputation of missing values - Experiment

This is a component for imputing missing values using mean, median or most frequent. It makes use of an implementation from [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use the [SDK](https://platiagro.github.io/sdk/) to load datasets, save models and other artifacts.
- how to declare parameters and use them to build reusable components.

## Declare parameters and model hyperparameters
Components may declare (and use) these default parameters:
- dataset
- target

Use these parameters to load/save datasets, models, metrics, and figures with the help of [PlatIAgro SDK](https://platiagro.github.io/sdk/). <br />
You may also declare custom parameters to set when running an experiment.

Select the hyperparameters and their respective values to be used when training the model:
- strategy

These parameters are just a few offered by the model class, you may also use another existing parameter. <br />
Check the [model parameters](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn-impute-simpleimputer) for more information.

In [ ]:
# parameters
dataset = "iris" #@param {type:"string"}
target = "Species" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

# hyperparameters
strategy_num = "mean" #@param ["mean", "median", "most_frequent", "constant"] {type:"string", label:"Estratégia de atribuição de valores para colunas numéricas", description:"'mean' é apenas para atributos numéricos, 'median' é apenas para atributos numéricos, 'most_frequent' é para atributos numéricos e categóricos e 'constant' substituirá os valores ausentes por fill_value}
strategy_cat = "most_frequent" #@param ["most_frequent", "constant"] {type:"string", label:"Estratégia de atribuição de valores para colunas categóricas", description:"'most_frequent' é para atributos numéricos e categóricos e 'constant' substituirá os valores ausentes por fill_value}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1)
y = df[target]

## Load metadata about the dataset
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Features configuration

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]

## Fit a model using sklearn.impute.SimpleImputer

In [ ]:
import pandas as pd
from platiagro.featuretypes import NUMERICAL
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline


pipeline = make_pipeline(
    make_column_transformer((SimpleImputer(strategy=strategy_num), numerical_indexes),
                            (SimpleImputer(strategy=strategy_cat), non_numerical_indexes),
                            remainder='passthrough'))

# Train model and transform dataset
X = pipeline.fit_transform(X)

features_after_pipeline = \
        np.concatenate((columns[numerical_indexes],
                        columns[non_numerical_indexes]))

# Put data back in a pandas.DataFrame
df = pd.DataFrame(data=X, columns=features_after_pipeline)
df[target] = y

## Save dataset

Stores the transformed dataset in a object storage.<br>

In [ ]:
from platiagro import save_dataset

save_dataset(name=dataset, df=df)

## Save model

Stores the model artifacts in a object storage.<br>
It will make the model available for future deployments.

In [ ]:
from platiagro import save_model

save_model(pipeline=pipeline,
           columns=columns,
           features_after_pipeline=features_after_pipeline)